# Adding and removing tags

* https://github.com/aol/moloch/wiki/API#addtags

Sessions can be marked with custom *tags*, so afterward we could include or exclude these sessions by simple `tags == ` query. Like before, start by setting up modules and auth variables.

In [2]:
import requests
from requests.auth import HTTPDigestAuth
user="vagrant"
passwd="vagrant"
auth=HTTPDigestAuth(user, passwd)

Unline previous endpoints, we are not just pulling data. So we need to do HTTP POST with a comma separated list of tags in POST body.

In [11]:
query = {
    "expression": "protocols == dns && (dns.host == berylia.org || dns.host == www.facebook.com || dns.host == sysadminnid.tumblr.com)",
    "date": 1,
}
tags = [
    "traffic",
    "generated",
    "good"
]
tags = ",".join(tags)
resp = requests.post("http://192.168.10.13:8005/addTags", params=query, auth=auth, json={
    "tags": tags
})
print(resp.text)

{"success":true,"text":"Tags added successfully"}


Other queries can receive a different set of tags.

In [12]:
query = {
    "expression": "protocols == dns && (dns.host == self-signed.badssl.com || dns.host == testmyids.com)",
    "date": 1,
}
tags = [
    "traffic",
    "generated",
    "bad"
]
tags = ",".join(tags)
resp = requests.post("http://192.168.10.13:8005/addTags", params=query, auth=auth, json={
    "tags": tags
})
print(resp.text)

{"success":true,"text":"Tags added successfully"}


Now we no longer have to write the complex query when looking for specific traffic subset.

In [13]:
query = {
    "expression": "tags == generated && tags == good",
    "exp": "dns.host"
}
resp = requests.get("http://192.168.10.13:8005/unique.txt", params=query, auth=auth)
print(resp.text)

berylia.org
star-mini.c10r.facebook.com
www.facebook.com
sysadminnid.tumblr.com



In [14]:
query = {
    "expression": "tags == generated && tags == bad",
    "exp": "dns.host"
}
resp = requests.get("http://192.168.10.13:8005/unique.txt", params=query, auth=auth)
print(resp.text)

testmyids.com
self-signed.badssl.com



Likewise, tags can be removed. For example, `traffic` tag is quite useless currently. Make sure that your API user has permissions to remove data though.

In [17]:
query = {
    "expression": "tags == traffic",
    "date": 1,
}
resp = requests.post("http://192.168.10.13:8005/removeTags", params=query, auth=auth, json={
    "tags": "traffic"
})

In [18]:
print(resp.text)

{"success":true,"text":"Tags removed successfully"}


# Tasks

Consider the suricata `community_id` task from last notebook. Write a script that extracts suricata Signature and Severity fields from Alert JSON. Tag moloch sessions that have corresponding Suricata IDS alerts with this information.
* Whitespace in signature name will not fly, replace that with underscore or dash;
* Likewise, consider lowercasing the signature name;
* A simple severity value number as tag is too vague, actual value in Moloch should be `suri-prio-1`, `suri-prio-2` and `suri-prio-3`;
* If using classroom server, prepend your name in front of the tag;